In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error

path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-11 20:55:04,327 utils 400 [INFO]    [logger_func] start 


In [225]:
#========================================================================
# CV CHECKER 
# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target, col_term]].set_index(key)
train = base[~base[target].isnull()]
base_idx = train.reset_index()[[key, target]]

check_model_path = '../stack/0208_002_stack_lgb_lr0.01_235feats_multi157_valterm_5seed_4leaves_iter1031_TERM24_CV3-631440687534257_LB.gz'
check_model = utils.read_pkl_gzip(check_model_path)[[key, 'pred_mean']].set_index(key)

# Indexをそろえる
train = train.join(check_model)

y_train = train[target].values
check_pred = train['pred_mean'].values
check_score = np.sqrt(mean_squared_error(y_train, check_pred))
print(check_score)
#========================================================================


100%|██████████| 3/3 [00:00<00:00, 56.44it/s]

3.7031587732980125


In [ ]:
#========================================================================
# Corr Checker
from itertools import combinations
base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
ens_list = glob.glob('../ensemble/*.gz')
ens_list = glob.glob('../out_ensemble/*.gz')
combi_path = combinations(ens_list, 2)

#========================================================================
# Classifier
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
clf['pred_mean_2'] = clf_2['pred_mean']
clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
base['clf_pred'] = clf['clf_pred']
base = base[base['clf_pred']<0.0072]
base = base[base[target]>-30]
#========================================================================

#========================================================================
# 相関
for p_1, p_2 in combi_path:
    try:
        m1 = utils.read_pkl_gzip(p_1)[[key, 'pred_mean']].set_index(key)
        base['p1'] = m1['pred_mean']
    except KeyError:
        m1 = utils.read_pkl_gzip(p_1)[[key, 'prediction']].set_index(key)
        base['p1'] = m1['prediction']
    try:
        m2 = utils.read_pkl_gzip(p_2)[[key, 'pred_mean']].set_index(key)
        base['p2'] = m2['pred_mean']
    except KeyError:
        m2 = utils.read_pkl_gzip(p_2)[[key, 'prediction']].set_index(key)
        base['p2'] = m2['prediction']
    if p_1.count('LB'):
        cv1 = re.search(r'CV([^/.]*)_LB.gz', p_1).group(1)
    else:
        cv1 = re.search(r'CV([^/.]*)', p_1.replace('.', '-')).group(1)
    if p_2.count('LB'):
        cv2 = re.search(r'CV([^/.]*)_LB.gz', p_2).group(1)
    else:
        cv2 = re.search(r'CV([^/.]*)', p_2.replace('.', '-')).group(1)
        
    corr = np.corrcoef(base['p1'], base['p2'].values).min()
    logger.info(f"CORR: {corr} | CV{cv1[:6]} vs CV{cv2[:6]}")
#========================================================================

In [15]:
# Data Load
# Model Load
base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
ens_list = glob.glob('../ensemble/*.gz')
for path in ens_list:
    try:
        cv = re.search(r'CV([^/.]*)_LB.gz', path).group(1)
    except AttributeError:
        cv = re.search(r'CV([^/.]*).gz', path).group(1)
    
    try:
        blend = utils.read_df_pkl(path).set_index(key)['pred_mean']
    except KeyError:
        blend = utils.read_df_pkl(path).set_index(key)['prediction']
    base[f'base_{cv}'] = blend
#========================================================================

out_list = glob.glob('../out_ensemble/*.gz')
out_list = ['../out_ensemble/0208_215_stack_lgb_lr0.01_236feats_10seed_70leaves_iter1908_OUT0_CV8-413966707443706_LB.gz']
for path in out_list:
    cv = re.search(r'CV([^/.]*)_LB.gz', path).group(1)
    blend = utils.read_pkl_gzip(path).set_index(key)['pred_mean']
    base[f"out_{cv}"] = blend

#========================================================================
# Classifier
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
clf['pred_mean_2'] = clf_2['pred_mean']
clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
# clf['clf_pred'] =  clf['pred_mean']
#========================================================================

# Indexをそろえる
base['clf_pred'] = clf['clf_pred']
base_cols = [col for col in base.columns if col.count('base_')]
out_cols = [col for col in base.columns if col.count('out_')]
base_cols = sorted(base_cols)
base = base[base_cols + [target, 'clf_pred'] + out_cols]
base.head()

100%|██████████| 1/1 [00:00<00:00,  3.22it/s]


,base_3-6215750935280235,base_3-6406114043915188,base_3-6437143344456357,base_3-643944732150466,base_3-6745325015227253,base_3-7005383169300727,base_3-7421434632512685,target,clf_pred,out_8-413966707443706
card_id,,,,,,,,,,
C_ID_92a2005557,-0.348066,-0.308465,-0.189379,-0.275469,-0.442548,-0.440412,-0.386080,-0.820283,0.000444,-0.349654
C_ID_3d0044924f,-0.433948,-0.501848,-0.276453,-0.512262,0.526564,0.191130,0.641302,0.392913,0.007831,0.261769
C_ID_d639edf6cd,0.707558,0.570250,0.586058,0.615076,0.499075,0.316775,0.691890,0.688056,0.004074,0.725757
C_ID_186d6a6901,0.160033,0.129957,0.128617,0.029858,0.316660,0.343618,0.301038,0.142495,0.000797,0.175317
C_ID_cdbd2c0db2,-0.110775,-0.293786,-0.008663,-0.045465,-0.142978,-0.133101,-0.081131,-0.159749,0.000251,-0.239419


In [163]:
#========================================================================
# Simple Blender 
min_thres = 0.0072
pred = 'prediction'

base[pred] = base[base_cols[0]] * 0.3 + base[base_cols[1]] * 0.15 + base[base_cols[2]] * 0.15 + base[base_cols[3]] * 0.15 + base[base_cols[4]] * 0.15 + base[base_cols[5]] * 0.05 + base[base_cols[6]] * 0.05
base[pred] = base[base_cols[0]] * 0.5 + base[base_cols[1]] * 0.1 + base[base_cols[2]] * 0.1 + base[base_cols[3]] * 0.10 + base[base_cols[4]] * 0.2
# base[pred] = base[base_cols[0]] * 1.0 + base[base_cols[4]] * 1.0
# base[pred] = base[base_cols[4]] * 1.5
base.loc[base['clf_pred']< min_thres, pred] = base.loc[base['clf_pred']< min_thres, out_cols[0]]
# base.loc[base['clf_pred']>=min_thres, pred] = base.loc[base['clf_pred']>=min_thres, pred]

train = base[~base[target].isnull()]
y_pred = train[pred].values
y_train = train[target].values
score = np.sqrt(mean_squared_error(y_train, y_pred))
print(score)

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit.to_csv(f'../submit/{start_time[4:12]}_submit_outlier_processing_blend{len(base_cols)}_out_blend{len(out_cols)}_minthres{min_thres}_CV{score}_LB')
display(submit.head())
#========================================================================

3.607750314545443


,target
card_id,
C_ID_0ab67a22ab,-1.455610
C_ID_130fd0cbdd,-0.171461
C_ID_b709037bc5,-1.243021
C_ID_d27d835a9f,-0.059027
C_ID_2b5e3df5c2,-1.361652


In [4]:
#========================================================================
# Manual PostProcessing
# Setting
# upper_1. Best Score: 3.6114026721733525 | Best Manual: 25.319999999999993 | Max Thres: 0.511
# upper_2. Best Score: 3.6105940647875325 | Best Manual: 9.119999999999997 | Max Thres: 0.471
#========================================================================

# Base Model Pred
base = utils.read_df_pkl('../input/base_term*')[[key, target, 'no_out_flg']].set_index(key)
out_list = ['../out_ensemble/0208_215_stack_lgb_lr0.01_236feats_10seed_70leaves_iter1908_OUT0_CV8-413966707443706_LB.gz']
out_model = utils.read_pkl_gzip('../out_ensemble/0211_020_elo_NN_stack_E1_linear1_235feat_const1_lr0.001_batch128_epoch30_CV1.5006922746209812.gz')[[key, 'prediction']].set_index(key)
# out_model = utils.read_pkl_gzip(out_list[0])[[key, 'prediction']].set_index(key)
best_model = utils.read_pkl_gzip('../ensemble/0206_125_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1164_OUT29.8269_CV3-6215750935280235_LB.gz')[[key, 'pred_mean']].set_index(key)
base['pred_mean'] = best_model['pred_mean']
base['no_out_pred'] = out_model['prediction']
# No Outlier Model の予測の欠損はAll Modelで埋める
base.loc[base['no_out_pred'].isnull(), 'no_out_pred'] = base.loc[base['no_out_pred'].isnull(), 'pred_mean']
#========================================================================
# Classifier
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
clf['pred_mean_2'] = clf_2['pred_mean']
clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
base['clf_pred'] = clf['clf_pred']
#========================================================================

is_manual = [True, False][1]
score_list = []
ratio_list = []
min_score = 100
pred = 'prediction'

train = base[~base[target].isnull()]
y_train = train[target].values
train[pred] = 0
# train[pred] = train['no_out_pred']
min_thres = 0.0072
upper_thres = 0.511
upper_value = -25.32
for min_thres in np.arange(0.005, 0.01, 0.0002):
# for max_thres in np.arange(0.47, 0.48, 0.001):
    
    length = train.loc[train['clf_pred']> min_thres].shape[0]
    
    # Under PostProcessing
    train.loc[train['clf_pred']< min_thres, pred] = train.loc[train['clf_pred']< min_thres, 'no_out_pred']
    train.loc[train['clf_pred']>=min_thres, pred] = train.loc[train['clf_pred']>=min_thres, 'pred_mean']
    train.loc[train['no_out_flg']==1, pred] =  train.loc[train['no_out_flg']==1, 'no_out_pred']

    # 閾値以上のtargetをマニュアル設定する
    if is_manual:
        tmp_pred = train[pred].values
        for manual in np.arange(9.0, 10, 0.02):
            train[pred] = tmp_pred.copy()
            train.loc[train['clf_pred']> max_thres, pred] = manual*-1
            # Upper PosrProcessing
            train.loc[train['clf_pred']> upper_thres, pred] = upper_value
    
            y_pred = train[pred].values
        
            score = np.sqrt(mean_squared_error(y_train, y_pred))
            if score<min_score:
                min_score = score
                best_thres = max_thres
                best_value = manual
                score_list.append(min_score)
                ratio_list.append(best_thres)
            if score<3.611:
#                 print(f"Best Score: {min_score} | Min Thres: {best_thres} | Max Thres: {max_thres} | MANUAL: {-1*manual}")
                print(f"Score: {score} | Best Score: {min_score} | Best Manual: {best_value} | Max Thres: {best_thres}")
        
    else:
        y_pred = train[pred].values
    
        score = np.sqrt(mean_squared_error(y_train, y_pred))
        if score<min_score:
            min_score = score
            best_thres = min_thres
            score_list.append(min_score)
            ratio_list.append(best_thres)
        print(f"Best Score: {min_score} | Best Thres: {best_thres} | LENGTH: {length}")

100%|██████████| 3/3 [00:00<00:00, 29.81it/s]


Best Score: 3.6013900197179627 | Best Thres: 0.005 | LENGTH: 55634
Best Score: 3.6010765270356484 | Best Thres: 0.0052 | LENGTH: 54160
Best Score: 3.6007401246202626 | Best Thres: 0.005399999999999999 | LENGTH: 52757
Best Score: 3.6005112264580323 | Best Thres: 0.005599999999999999 | LENGTH: 51425
Best Score: 3.600127569614325 | Best Thres: 0.005799999999999999 | LENGTH: 50151
Best Score: 3.599886486980757 | Best Thres: 0.005999999999999998 | LENGTH: 48984
Best Score: 3.599545893053644 | Best Thres: 0.006199999999999998 | LENGTH: 47823
Best Score: 3.5992515028394383 | Best Thres: 0.006399999999999998 | LENGTH: 46760
Best Score: 3.5990798786569904 | Best Thres: 0.006599999999999997 | LENGTH: 45788
Best Score: 3.5987722757865277 | Best Thres: 0.006799999999999997 | LENGTH: 44789
Best Score: 3.5985610746031376 | Best Thres: 0.006999999999999997 | LENGTH: 43873
Best Score: 3.598244430879445 | Best Thres: 0.007199999999999996 | LENGTH: 43043
Best Score: 3.598089051063766 | Best Thres: 0.007

In [8]:
utils.to_pkl_gzip(obj=base.reset_index()[[key, 'clf_pred']], path='../input/base_clf')

In [15]:
blend_num = 3
out_blend_num = 2
thres = 0.0072
base.loc[base['clf_pred']< thres, pred] = base.loc[base['clf_pred']< thres, 'no_out_pred']
base.loc[base['clf_pred']>=thres, pred] = base.loc[base['clf_pred']>=thres, 'pred_mean']
base.loc[base['no_out_flg']==1, pred] =  base.loc[base['no_out_flg']==1, 'no_out_pred']
display(base.head())

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit.to_csv(f'../submit/{start_time[4:12]}_submit_outlier_processing_blend{blend_num}_0.4_0.3_0.3_out_blend{out_blend_num}_0.7_0.3_CV{min_score}_LB')
display(submit.head())

,target,no_out_flg,pred_mean,p1,p2,p3,no_out_pred,clf_pred,prediction
card_id,,,,,,,,,
C_ID_92a2005557,-0.820283,NaN,-0.312293,-0.277248,-0.299641,-0.295437,-0.339001,0.000444,-0.339001
C_ID_3d0044924f,0.392913,NaN,-0.402629,-0.464311,-0.299189,-0.659761,0.276268,0.007831,-0.402629
C_ID_d639edf6cd,0.688056,NaN,0.740270,0.764349,0.759806,0.823495,0.770552,0.004074,0.770552
C_ID_186d6a6901,0.142495,NaN,0.134626,0.101346,0.134031,0.057771,0.170184,0.000797,0.170184
C_ID_cdbd2c0db2,-0.159749,1.0,-0.184077,-0.222948,-0.242942,-0.158849,-0.238870,0.000251,-0.238870


,target
card_id,
C_ID_0ab67a22ab,-1.659109
C_ID_130fd0cbdd,-0.203953
C_ID_b709037bc5,-1.298058
C_ID_d27d835a9f,-0.052482
C_ID_2b5e3df5c2,-1.371860


In [36]:
from itertools import product, permutations
#========================================================================
# Randomness Blendのスコアを見る
#========================================================================
score_list = []
best_ratio_list = []
min_score = 100
train = base[~base[target].isnull()]
y_train = train[target].values


cnt = 0
while cnt!=10000:
    cnt+=1
    np.random.seed(cnt)
    ratio_list = np.arange(0.05, 1.0, 0.05)
    np.random.seed(cnt+1); np.random.shuffle(ratio_list)
    r1 = ratio_list.copy()
    np.random.seed(cnt*2); np.random.shuffle(ratio_list)
    r2 = ratio_list.copy()
    np.random.seed(cnt*3); np.random.shuffle(ratio_list)
    r3 = ratio_list.copy()
    np.random.seed(cnt*4); np.random.shuffle(ratio_list)
    r4 = ratio_list.copy()
    ratio_combi = zip(r1, r2, r3, r4)
    
    for i, ratio in enumerate(ratio_combi):
        
        p1 = train[base_cols[0]].values
        p2 = train[base_cols[1]].values
        p3 = train[base_cols[2]].values
        p4 = train[base_cols[3]].values
#         if cnt==1:
#             print(
#                 np.sqrt(mean_squared_error(y_train, p1))
#                 ,np.sqrt(mean_squared_error(y_train, p2))
#                 ,np.sqrt(mean_squared_error(y_train, p3))
#                 ,np.sqrt(mean_squared_error(y_train, p4))
#             )
#             sys.exit()
        
        ratio = np.array(ratio)
        r_sum = ratio.sum()
        ratio /= r_sum
        a = ratio[0]
        b = ratio[1]
        c = ratio[2]
        d = ratio[3]
        coef_list = [a,b,c,d]
        
        combi_list = permutations(coef_list)
        for combi in combi_list:
            e = combi[0]
            f = combi[1]
            g = combi[2]
            h = combi[3]
            y_pred = p1*e + p2*f + p3*g + p4*h
            
            score = np.sqrt(mean_squared_error(y_train, y_pred))
            if score<min_score:
                min_score = score
                min_ratio = combi
                
                score_list.append(min_score)
                best_ratio_list.append(min_ratio)
        
    if cnt%200==0:
        print(f"BEST: {np.min(score_list)} | BEST RATIO: {best_ratio_list[np.argmin(score_list)]}")
        
print("Complete!!")

BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454545456)
BEST: 3.6168798285416592 | BEST RATIO: (0.045454545454545456, 0.045454545454545456, 0.8636363636363636, 0.045454545454

KeyboardInterrupt: 